# Financial Analysis - Teste e Validação

Notebook para testar e validar cálculos financeiros avançados.
Baseado no código do pvlib-service.

In [ ]:
# Instalação das dependências necessárias
!pip install pandas numpy requests matplotlib seaborn jupyter ipython
!pip install pvlib-python scipy scikit-learn

# Verificar instalação
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todas as dependências foram instaladas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"🌐 Requests disponível")
print(f"📈 Matplotlib disponível")
print(f"🎨 Seaborn disponível")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configurações de plotagem
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [14, 10]
plt.rcParams['font.size'] = 10

## 📥 Carregar Dados dos Notebooks Anteriores

In [ ]:
# Carregar dados dos notebooks anteriores
try:
    # Dados PVGIS
    with open('../data/pvgis_monthly_data.json', 'r') as f:
        pvgis_data = json.load(f)
    
    # Dados de módulos
    with open('../data/module_calculation_results.json', 'r') as f:
        module_data = json.load(f)
    
    # Configuração
    with open('../data/config.json', 'r') as f:
        config = json.load(f)
    
    print("✅ Dados carregados com sucesso!")
    print(f"📊 Geração anual: {module_data['sistema']['energia_total_anual_kwh']:,.1f} kWh")
    print(f"⚡ Potência: {module_data['sistema']['potencia_total_kw']:.2f} kWp")
    print(f"🔋 Módulos: {module_data['sistema']['num_modulos']}")
    
except FileNotFoundError:
    print("❌ Execute primeiro os notebooks 01 e 02")
    raise

## 💰 Dados de Entrada Financeiros
Defina aqui os parâmetros financeiros:

In [ ]:
# PARÂMETROS FINANCEIROS - EDITE AQUI

# Investimento e custos
CUSTO_POR_KWP = 4500  # R$/kWp instalado
INVESTIMENTO_INICIAL = module_data['sistema']['potencia_total_kw'] * CUSTO_POR_KWP

# Geração e consumo (do notebook anterior)
GERACAO_MENSAL = module_data['sistema']['energia_mensal']  # kWh/mês
CONSUMO_MENSAL = [module_data['configuracao']['consumo_anual_kwh'] / 12] * 12  # kWh/mês

# Tarifas de energia
TARIFA_ENERGIA = 0.85  # R$/kWh
CUSTO_FIO_B = 0.30  # R$/kWh (custo do fio B)

# Parâmetros temporais e inflação
VIDA_UTIL = 25  # anos
TAXA_DESCONTO = 8.0  # % ao ano (SELIC)
INFLACAO_ENERGIA = 4.5  # % ao ano

# Parâmetros opcionais
DEGRADACAO_MODULOS = 0.5  # % ao ano
CUSTO_OM_ANUAL = INVESTIMENTO_INICIAL * 0.01  # 1% ao ano
INFLACAO_OM = 4.0  # % ao ano

print("💰 CONFIGURAÇÃO FINANCEIRA:")
print(f"💸 Investimento inicial: R$ {INVESTIMENTO_INICIAL:,.2f}")
print(f"💡 Tarifa energia: R$ {TARIFA_ENERGIA:.2f}/kWh")
print(f"🔌 Custo fio B: R$ {CUSTO_FIO_B:.2f}/kWh")
print(f"📈 Taxa desconto: {TAXA_DESCONTO:.1f}%")
print(f"📊 Inflação energia: {INFLACAO_ENERGIA:.1f}%")
print(f"🛠️ Custo O&M anual: R$ {CUSTO_OM_ANUAL:,.2f}")
print(f"📉 Degradação: {DEGRADACAO_MODULOS:.1f}%/ano")
print(f"⏱️ Vida útil: {VIDA_UTIL} anos")

## 🛠️ Funções Financeiras (extraídas do pvlib-service)

In [ ]:
class FinancialCalculationService:
    """Serviço para cálculos financeiros avançados - Baseado no pvlib-service"""
    
    def __init__(self):
        pass
    
    def calculate_advanced_financials(self, input_data: Dict) -> Dict:
        """Calcular análise financeira completa"""
        
        print("🚀 Iniciando análise financeira avançada...")
        print(f"💸 Investimento: R$ {input_data['investimento_inicial']:,.2f}")
        print(f"⚡ Geração anual: {sum(input_data['geracao_mensal']):,.1f} kWh")
        
        # 1. Calcular fluxo de caixa detalhado
        cash_flow = self._calculate_detailed_cash_flow(input_data)
        
        # 2. Calcular indicadores principais
        vpl = self._calculate_npv(cash_flow, input_data['taxa_desconto']) - input_data['investimento_inicial']
        tir = self._calculate_irr(cash_flow, input_data['investimento_inicial'])
        payback_simples = self._calculate_simple_payback(cash_flow)
        payback_descontado = self._calculate_discounted_payback(cash_flow, input_data['taxa_desconto'])
        
        # 3. Métricas adicionais
        geracao_anual_inicial = sum(input_data['geracao_mensal'])
        economia_total_25_anos = sum(year['economia_energia'] for year in cash_flow)
        economia_anual_media = economia_total_25_anos / input_data['vida_util']
        lucratividade_index = (vpl + input_data['investimento_inicial']) / input_data['investimento_inicial']
        
        # 4. Indicadores de performance
        indicadores = self._calculate_performance_indicators(
            input_data, cash_flow, geracao_anual_inicial
        )
        
        # 5. Análise de sensibilidade
        sensibilidade = self._calculate_sensitivity_analysis(input_data)
        
        # 6. Análise de cenários
        cenarios = self._calculate_scenario_analysis(input_data)
        
        resultado = {
            'vpl': round(vpl, 2),
            'tir': round(tir, 2),
            'payback_simples': round(payback_simples, 2),
            'payback_descontado': round(payback_descontado, 2),
            'economia_total_25_anos': round(economia_total_25_anos, 2),
            'economia_anual_media': round(economia_anual_media, 2),
            'lucratividade_index': round(lucratividade_index, 3),
            'cash_flow': cash_flow,
            'indicadores': indicadores,
            'sensibilidade': sensibilidade,
            'cenarios': cenarios
        }
        
        print("✅ Análise financeira concluída!")
        return resultado
    
    def _calculate_detailed_cash_flow(self, input_data: Dict) -> List[Dict]:
        """Calcular fluxo de caixa detalhado ano a ano"""
        
        cash_flow = []
        geracao_anual_inicial = sum(input_data['geracao_mensal'])
        consumo_anual = sum(input_data['consumo_mensal'])
        fluxo_acumulado = -input_data['investimento_inicial']
        
        for ano in range(1, input_data['vida_util'] + 1):
            # Geração com degradação
            fator_degradacao = (1 - input_data['degradacao_modulos'] / 100) ** (ano - 1)
            geracao_anual = geracao_anual_inicial * fator_degradacao
            
            # Inflação da tarifa
            fator_inflacao_energia = (1 + input_data['inflacao_energia'] / 100) ** (ano - 1)
            tarifa_ajustada = input_data['tarifa_energia'] * fator_inflacao_energia
            custo_fio_ajustado = input_data['custo_fio_b'] * fator_inflacao_energia
            
            # Calcular economia
            economia_energia = self._calculate_energy_savings(
                geracao_anual, consumo_anual, tarifa_ajustada, custo_fio_ajustado
            )
            
            # Custos O&M
            fator_inflacao_om = (1 + input_data['inflacao_om'] / 100) ** (ano - 1)
            custos_om = input_data['custo_om'] * fator_inflacao_om
            
            # Fluxo líquido
            fluxo_liquido = economia_energia - custos_om
            fluxo_acumulado += fluxo_liquido
            
            # Valor presente
            valor_presente = fluxo_liquido / ((1 + input_data['taxa_desconto'] / 100) ** ano)
            
            cash_flow.append({
                'ano': ano,
                'geracao_anual': round(geracao_anual, 1),
                'economia_energia': round(economia_energia, 2),
                'custos_om': round(custos_om, 2),
                'fluxo_liquido': round(fluxo_liquido, 2),
                'fluxo_acumulado': round(fluxo_acumulado, 2),
                'valor_presente': round(valor_presente, 2)
            })
        
        return cash_flow
    
    def _calculate_energy_savings(self, geracao: float, consumo: float, 
                                 tarifa: float, custo_fio: float) -> float:
        """Calcular economia com energia"""
        if geracao >= consumo:
            # Geração maior que consumo
            energia_injetada = geracao - consumo
            economia = consumo * tarifa + energia_injetada * (tarifa - custo_fio)
        else:
            # Geração menor que consumo
            economia = geracao * tarifa
        
        return economia
    
    def _calculate_npv(self, cash_flow: List[Dict], taxa_desconto: float) -> float:
        """Calcular VPL"""
        return sum(year['valor_presente'] for year in cash_flow)
    
    def _calculate_irr(self, cash_flow: List[Dict], investimento_inicial: float) -> float:
        """Calcular TIR usando Newton-Raphson"""
        
        fluxos = [-investimento_inicial] + [year['fluxo_liquido'] for year in cash_flow]
        taxa = 0.1  # Estimativa inicial
        
        for _ in range(100):
            vpl = sum(fluxo / ((1 + taxa) ** i) for i, fluxo in enumerate(fluxos))
            dvpl = sum(-i * fluxo / ((1 + taxa) ** (i + 1)) for i, fluxo in enumerate(fluxos))
            
            if abs(vpl) < 1e-6:
                break
            
            if abs(dvpl) < 1e-10:
                break
            
            taxa_nova = taxa - vpl / dvpl
            
            if abs(taxa_nova - taxa) < 1e-6:
                break
            
            taxa = taxa_nova
        
        return taxa * 100
    
    def _calculate_simple_payback(self, cash_flow: List[Dict]) -> float:
        """Calcular payback simples"""
        for year in cash_flow:
            if year['fluxo_acumulado'] >= 0:
                if year['ano'] > 1:
                    year_anterior = cash_flow[year['ano'] - 2]
                    fator = abs(year_anterior['fluxo_acumulado']) / year['fluxo_liquido']
                    return year['ano'] - 1 + fator
                else:
                    return float(year['ano'])
        
        return float('inf')
    
    def _calculate_discounted_payback(self, cash_flow: List[Dict], taxa_desconto: float) -> float:
        """Calcular payback descontado"""
        vpl_acumulado = 0
        
        for year in cash_flow:
            vpl_acumulado += year['valor_presente']
            
            if vpl_acumulado >= 0:
                if year['ano'] > 1:
                    vpl_anterior = vpl_acumulado - year['valor_presente']
                    fator = abs(vpl_anterior) / year['valor_presente']
                    return year['ano'] - 1 + fator
                else:
                    return float(year['ano'])
        
        return float('inf')
    
    def _calculate_performance_indicators(self, input_data: Dict, 
                                        cash_flow: List[Dict], 
                                        geracao_anual_inicial: float) -> Dict:
        """Calcular indicadores de performance"""
        
        # Yield específico
        yield_especifico = geracao_anual_inicial / (input_data['investimento_inicial'] / 1000)
        
        # LCOE
        custo_nivelado_energia = self._calculate_lcoe(
            input_data['investimento_inicial'], cash_flow, input_data['taxa_desconto']
        )
        
        # Eficiência do investimento
        economia_anual_media = sum(year['economia_energia'] for year in cash_flow) / input_data['vida_util']
        eficiencia_investimento = (economia_anual_media / input_data['investimento_inicial']) * 100
        
        # ROI
        economia_total = sum(year['economia_energia'] for year in cash_flow)
        retorno_sobre_investimento = ((economia_total - input_data['investimento_inicial']) / input_data['investimento_inicial']) * 100
        
        return {
            'yield_especifico': round(yield_especifico, 1),
            'custo_nivelado_energia': round(custo_nivelado_energia, 4),
            'eficiencia_investimento': round(eficiencia_investimento, 1),
            'retorno_sobre_investimento': round(retorno_sobre_investimento, 1)
        }
    
    def _calculate_lcoe(self, investimento: float, cash_flow: List[Dict], taxa_desconto: float) -> float:
        """Calcular LCOE"""
        energia_descontada = sum(
            year['geracao_anual'] / ((1 + taxa_desconto / 100) ** year['ano'])
            for year in cash_flow
        )
        
        if energia_descontada == 0:
            return 0
        
        return investimento / energia_descontada
    
    def _calculate_sensitivity_analysis(self, input_data: Dict) -> Dict:
        """Análise de sensibilidade"""
        
        # Variação da tarifa
        vpl_tarifa = []
        for variacao in range(-20, 25, 5):
            input_variado = input_data.copy()
            input_variado['tarifa_energia'] = input_data['tarifa_energia'] * (1 + variacao / 100)
            
            cash_flow_variado = self._calculate_detailed_cash_flow(input_variado)
            vpl_variado = self._calculate_npv(cash_flow_variado, input_data['taxa_desconto']) - input_data['investimento_inicial']
            
            vpl_tarifa.append({
                'parametro': input_variado['tarifa_energia'],
                'vpl': round(vpl_variado, 2)
            })
        
        # Variação da inflação
        vpl_inflacao = []
        for variacao in np.arange(-2, 2.5, 0.5):
            input_variado = input_data.copy()
            input_variado['inflacao_energia'] = input_data['inflacao_energia'] + variacao
            
            cash_flow_variado = self._calculate_detailed_cash_flow(input_variado)
            vpl_variado = self._calculate_npv(cash_flow_variado, input_data['taxa_desconto']) - input_data['investimento_inicial']
            
            vpl_inflacao.append({
                'parametro': input_variado['inflacao_energia'],
                'vpl': round(vpl_variado, 2)
            })
        
        # Variação da taxa de desconto
        vpl_desconto = []
        for variacao in np.arange(-2, 2.5, 0.5):
            input_variado = input_data.copy()
            input_variado['taxa_desconto'] = input_data['taxa_desconto'] + variacao
            
            cash_flow_variado = self._calculate_detailed_cash_flow(input_variado)
            vpl_variado = self._calculate_npv(cash_flow_variado, input_variado['taxa_desconto']) - input_data['investimento_inicial']
            
            vpl_desconto.append({
                'parametro': input_variado['taxa_desconto'],
                'vpl': round(vpl_variado, 2)
            })
        
        return {
            'vpl_variacao_tarifa': vpl_tarifa,
            'vpl_variacao_inflacao': vpl_inflacao,
            'vpl_variacao_desconto': vpl_desconto
        }
    
    def _calculate_scenario_analysis(self, input_data: Dict) -> Dict:
        """Análise de cenários"""
        
        def calculate_basic_indicators(data: Dict):
            cash_flow = self._calculate_detailed_cash_flow(data)
            vpl = self._calculate_npv(cash_flow, data['taxa_desconto']) - data['investimento_inicial']
            tir = self._calculate_irr(cash_flow, data['investimento_inicial'])
            payback = self._calculate_simple_payback(cash_flow)
            return {"vpl": round(vpl, 2), "tir": round(tir, 2), "payback": round(payback, 2)}
        
        # Cenário base
        base_results = calculate_basic_indicators(input_data)
        
        # Cenário otimista
        input_otimista = input_data.copy()
        input_otimista['tarifa_energia'] *= 1.10
        input_otimista['taxa_desconto'] = max(1.0, input_otimista['taxa_desconto'] - 1.0)
        input_otimista['investimento_inicial'] *= 0.80
        otimista_results = calculate_basic_indicators(input_otimista)
        
        # Cenário conservador
        input_conservador = input_data.copy()
        input_conservador['tarifa_energia'] *= 0.95
        input_conservador['taxa_desconto'] += 1.0
        conservador_results = calculate_basic_indicators(input_conservador)
        
        # Cenário pessimista
        input_pessimista = input_data.copy()
        input_pessimista['tarifa_energia'] *= 0.90
        input_pessimista['taxa_desconto'] += 2.0
        input_pessimista['investimento_inicial'] *= 1.20
        pessimista_results = calculate_basic_indicators(input_pessimista)
        
        return {
            'base': base_results,
            'otimista': otimista_results,
            'conservador': conservador_results,
            'pessimista': pessimista_results
        }

## 💰 Executar Análise Financeira

In [ ]:
# Preparar dados de entrada
financial_input = {
    'investimento_inicial': INVESTIMENTO_INICIAL,
    'geracao_mensal': GERACAO_MENSAL,
    'consumo_mensal': CONSUMO_MENSAL,
    'tarifa_energia': TARIFA_ENERGIA,
    'custo_fio_b': CUSTO_FIO_B,
    'vida_util': VIDA_UTIL,
    'taxa_desconto': TAXA_DESCONTO,
    'inflacao_energia': INFLACAO_ENERGIA,
    'degradacao_modulos': DEGRADACAO_MODULOS,
    'custo_om': CUSTO_OM_ANUAL,
    'inflacao_om': INFLACAO_OM
}

print("📋 Dados de entrada preparados:")
for key, value in financial_input.items():
    if isinstance(value, list):
        print(f"{key}: {len(value)} valores (soma: {sum(value):,.1f})")
    elif isinstance(value, (int, float)):
        print(f"{key}: {value:,.2f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Executar análise financeira
financial_service = FinancialCalculationService()
resultado_financeiro = financial_service.calculate_advanced_financials(financial_input)

print("\n" + "="*60)
print("💰 RESULTADOS DA ANÁLISE FINANCEIRA")
print("="*60)
print(f"💎 VPL (25 anos): R$ {resultado_financeiro['vpl']:,.2f}")
print(f"📈 TIR: {resultado_financeiro['tir']:.2f}%")
print(f"⏱️ Payback Simples: {resultado_financeiro['payback_simples']:.1f} anos")
print(f"⏳ Payback Descontado: {resultado_financeiro['payback_descontado']:.1f} anos")
print(f"💰 Economia Total (25 anos): R$ {resultado_financeiro['economia_total_25_anos']:,.2f}")
print(f"📊 Economia Anual Média: R$ {resultado_financeiro['economia_anual_media']:,.2f}")
print(f"📋 Índice Lucratividade: {resultado_financeiro['lucratividade_index']:.3f}")

print("\n" + "="*60)
print("📊 INDICADORES DE PERFORMANCE")
print("="*60)
indicadores = resultado_financeiro['indicadores']
print(f"⚡ Yield Específico: {indicadores['yield_especifico']:.1f} kWh/R$1000")
print(f"💡 LCOE: R$ {indicadores['custo_nivelado_energia']:.4f}/kWh")
print(f"🎯 Eficiência Investimento: {indicadores['eficiencia_investimento']:.1f}%")
print(f"📈 ROI (25 anos): {indicadores['retorno_sobre_investimento']:.1f}%")

## 📊 Análise do Fluxo de Caixa

In [ ]:
# Criar DataFrame do fluxo de caixa
df_cashflow = pd.DataFrame(resultado_financeiro['cash_flow'])

print("💰 FLUXO DE CAIXA DETALHADO:")
print("\nPrimeiros 10 anos:")
display(df_cashflow.head(10)[['ano', 'geracao_anual', 'economia_energia', 'custos_om', 'fluxo_liquido', 'fluxo_acumulado']])

print("\nÚltimos 5 anos:")
display(df_cashflow.tail(5)[['ano', 'geracao_anual', 'economia_energia', 'custos_om', 'fluxo_liquido', 'fluxo_acumulado']])

# Estatísticas do fluxo de caixa
print("\n📊 ESTATÍSTICAS DO FLUXO DE CAIXA:")
print(f"💰 Fluxo médio anual: R$ {df_cashflow['fluxo_liquido'].mean():,.2f}")
print(f"📈 Maior fluxo anual: R$ {df_cashflow['fluxo_liquido'].max():,.2f} (Ano {df_cashflow.loc[df_cashflow['fluxo_liquido'].idxmax(), 'ano']})")
print(f"📉 Menor fluxo anual: R$ {df_cashflow['fluxo_liquido'].min():,.2f} (Ano {df_cashflow.loc[df_cashflow['fluxo_liquido'].idxmin(), 'ano']})")
print(f"📊 Desvio padrão: R$ {df_cashflow['fluxo_liquido'].std():,.2f}")

# Ano de break-even
breakeven_year = df_cashflow[df_cashflow['fluxo_acumulado'] >= 0]['ano'].iloc[0] if (df_cashflow['fluxo_acumulado'] >= 0).any() else None
if breakeven_year:
    print(f"⚖️ Break-even no ano: {breakeven_year}")
else:
    print("⚠️ Sistema não alcança break-even em 25 anos")

## 📈 Análise de Sensibilidade

In [ ]:
sensibilidade = resultado_financeiro['sensibilidade']

print("📊 ANÁLISE DE SENSIBILIDADE - VPL:")
print("\n🔸 Variação da Tarifa de Energia:")
for item in sensibilidade['vpl_variacao_tarifa'][::2]:  # Mostrar de 2 em 2
    variacao = (item['parametro'] / TARIFA_ENERGIA - 1) * 100
    print(f"   {variacao:+.0f}% (R$ {item['parametro']:.3f}/kWh) → VPL: R$ {item['vpl']:,.2f}")

print("\n🔸 Variação da Inflação Energética:")
for item in sensibilidade['vpl_variacao_inflacao'][::2]:  # Mostrar de 2 em 2
    print(f"   {item['parametro']:.1f}%/ano → VPL: R$ {item['vpl']:,.2f}")

print("\n🔸 Variação da Taxa de Desconto:")
for item in sensibilidade['vpl_variacao_desconto'][::2]:  # Mostrar de 2 em 2
    print(f"   {item['parametro']:.1f}%/ano → VPL: R$ {item['vpl']:,.2f}")

## 🎭 Análise de Cenários

In [ ]:
cenarios = resultado_financeiro['cenarios']

print("🎭 ANÁLISE DE CENÁRIOS:")
print("\n📊 Comparação dos Cenários:")
print(f"{'Cenário':<12} {'VPL (R$)':<15} {'TIR (%)':<10} {'Payback (anos)':<15}")
print("-" * 52)

for nome, dados in cenarios.items():
    print(f"{nome.capitalize():<12} {dados['vpl']:>12,.2f} {dados['tir']:>8.1f} {dados['payback']:>13.1f}")

# Análise de risco
vpls = [dados['vpl'] for dados in cenarios.values()]
vpl_medio = np.mean(vpls)
vpl_std = np.std(vpls)
coef_variacao = vpl_std / vpl_medio * 100 if vpl_medio != 0 else float('inf')

print(f"\n📈 ANÁLISE DE RISCO:")
print(f"💰 VPL médio: R$ {vpl_medio:,.2f}")
print(f"📊 Desvio padrão: R$ {vpl_std:,.2f}")
print(f"📋 Coeficiente de variação: {coef_variacao:.1f}%")
print(f"📉 Pior caso: R$ {min(vpls):,.2f} ({list(cenarios.keys())[vpls.index(min(vpls))]})")
print(f"📈 Melhor caso: R$ {max(vpls):,.2f} ({list(cenarios.keys())[vpls.index(max(vpls))]})")

## 📊 Visualizações Financeiras

In [ ]:
# Criar visualizações
fig = plt.figure(figsize=(18, 14))

# 1. Fluxo de Caixa Acumulado
ax1 = plt.subplot(2, 3, 1)
anos = df_cashflow['ano']
plt.plot(anos, df_cashflow['fluxo_acumulado'], marker='o', linewidth=3, color='green')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.title('Fluxo de Caixa Acumulado', fontsize=14, fontweight='bold')
plt.xlabel('Ano')
plt.ylabel('R$ (milhares)')
plt.grid(True, alpha=0.3)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}k'))

# 2. Economia vs Custos O&M
ax2 = plt.subplot(2, 3, 2)
plt.plot(anos, df_cashflow['economia_energia'], marker='s', label='Economia Energia', linewidth=2)
plt.plot(anos, df_cashflow['custos_om'], marker='^', label='Custos O&M', linewidth=2)
plt.title('Economia vs Custos O&M', fontsize=14, fontweight='bold')
plt.xlabel('Ano')
plt.ylabel('R$')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Geração com Degradação
ax3 = plt.subplot(2, 3, 3)
plt.plot(anos, df_cashflow['geracao_anual'], marker='o', color='orange', linewidth=2)
plt.title('Geração Anual com Degradação', fontsize=14, fontweight='bold')
plt.xlabel('Ano')
plt.ylabel('kWh/ano')
plt.grid(True, alpha=0.3)

# 4. Análise de Sensibilidade - Tarifa
ax4 = plt.subplot(2, 3, 4)
tarifa_data = sensibilidade['vpl_variacao_tarifa']
tarifas = [item['parametro'] for item in tarifa_data]
vpls_tarifa = [item['vpl'] for item in tarifa_data]
plt.plot(tarifas, vpls_tarifa, marker='o', linewidth=3, color='blue')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.axvline(x=TARIFA_ENERGIA, color='green', linestyle='--', alpha=0.7, label='Tarifa Base')
plt.title('Sensibilidade: VPL vs Tarifa', fontsize=14, fontweight='bold')
plt.xlabel('Tarifa (R$/kWh)')
plt.ylabel('VPL (R$)')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Análise de Cenários
ax5 = plt.subplot(2, 3, 5)
nomes_cenarios = list(cenarios.keys())
vpls_cenarios = [cenarios[nome]['vpl'] for nome in nomes_cenarios]
colors = ['blue', 'green', 'orange', 'red']
bars = plt.bar(nomes_cenarios, vpls_cenarios, color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.8)
plt.title('VPL por Cenário', fontsize=14, fontweight='bold')
plt.ylabel('VPL (R$)')
plt.xticks(rotation=45)

# Adicionar valores nas barras
for bar, valor in zip(bars, vpls_cenarios):
    height = bar.get_height()
    plt.annotate(f'R${valor/1000:.0f}k',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3 if height >= 0 else -15),
                textcoords="offset points",
                ha='center', va='bottom' if height >= 0 else 'top',
                fontweight='bold')

# 6. TIR por Cenário
ax6 = plt.subplot(2, 3, 6)
tirs_cenarios = [cenarios[nome]['tir'] for nome in nomes_cenarios]
bars = plt.bar(nomes_cenarios, tirs_cenarios, color=colors, alpha=0.7)
plt.axhline(y=TAXA_DESCONTO, color='red', linestyle='--', alpha=0.7, label=f'Taxa Desconto ({TAXA_DESCONTO}%)')
plt.title('TIR por Cenário', fontsize=14, fontweight='bold')
plt.ylabel('TIR (%)')
plt.xticks(rotation=45)
plt.legend()

# Adicionar valores nas barras
for bar, valor in zip(bars, tirs_cenarios):
    height = bar.get_height()
    plt.annotate(f'{valor:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Resumo final
print("\n" + "="*60)
print("🎯 RESUMO EXECUTIVO")
print("="*60)
if resultado_financeiro['vpl'] > 0:
    print("✅ INVESTIMENTO VIÁVEL:")
    print(f"   • VPL positivo de R$ {resultado_financeiro['vpl']:,.2f}")
    print(f"   • TIR de {resultado_financeiro['tir']:.1f}% > Taxa desconto ({TAXA_DESCONTO}%)")
    print(f"   • Payback em {resultado_financeiro['payback_simples']:.1f} anos")
else:
    print("❌ INVESTIMENTO NÃO VIÁVEL:")
    print(f"   • VPL negativo de R$ {resultado_financeiro['vpl']:,.2f}")
    print(f"   • TIR de {resultado_financeiro['tir']:.1f}% < Taxa desconto ({TAXA_DESCONTO}%)")

print(f"\n💰 Economia total em 25 anos: R$ {resultado_financeiro['economia_total_25_anos']:,.2f}")
print(f"📊 Retorno sobre investimento: {indicadores['retorno_sobre_investimento']:.0f}%")
print(f"⚡ LCOE: R$ {indicadores['custo_nivelado_energia']:.3f}/kWh")

## 💾 Salvar Resultados Financeiros

In [ ]:
# Salvar resultados completos
resultados_financeiros_completos = {
    'input': financial_input,
    'resultados': resultado_financeiro,
    'sistema_pv': {
        'potencia_kwp': module_data['sistema']['potencia_total_kw'],
        'num_modulos': module_data['sistema']['num_modulos'],
        'geracao_anual_kwh': module_data['sistema']['energia_total_anual_kwh']
    },
    'localizacao': config,
    'timestamp': pd.Timestamp.now().isoformat()
}

# Salvar em JSON
with open('../data/financial_analysis_results.json', 'w') as f:
    json.dump(resultados_financeiros_completos, f, indent=2, default=str)

# Salvar fluxo de caixa em CSV
df_cashflow.to_csv('../data/cash_flow_analysis.csv', index=False)

# Criar relatório resumido
relatorio_resumido = f"""
RELATÓRIO FINANCEIRO - SISTEMA FOTOVOLTAICO
============================================

DADOS DO SISTEMA:
• Potência instalada: {module_data['sistema']['potencia_total_kw']:.2f} kWp
• Número de módulos: {module_data['sistema']['num_modulos']}
• Geração anual: {sum(GERACAO_MENSAL):,.0f} kWh
• Consumo anual: {sum(CONSUMO_MENSAL):,.0f} kWh
• Cobertura: {module_data['sistema']['cobertura_percentual']:.1f}%

INVESTIMENTO:
• Investimento inicial: R$ {INVESTIMENTO_INICIAL:,.2f}
• Custo por kWp: R$ {CUSTO_POR_KWP:,.2f}
• O&M anual: R$ {CUSTO_OM_ANUAL:,.2f}

RESULTADOS FINANCEIROS:
• VPL (25 anos): R$ {resultado_financeiro['vpl']:,.2f}
• TIR: {resultado_financeiro['tir']:.2f}%
• Payback simples: {resultado_financeiro['payback_simples']:.1f} anos
• Payback descontado: {resultado_financeiro['payback_descontado']:.1f} anos
• Economia total (25 anos): R$ {resultado_financeiro['economia_total_25_anos']:,.2f}
• ROI: {indicadores['retorno_sobre_investimento']:.0f}%
• LCOE: R$ {indicadores['custo_nivelado_energia']:.4f}/kWh

ANÁLISE DE CENÁRIOS:
• Base: VPL R$ {cenarios['base']['vpl']:,.2f} | TIR {cenarios['base']['tir']:.1f}%
• Otimista: VPL R$ {cenarios['otimista']['vpl']:,.2f} | TIR {cenarios['otimista']['tir']:.1f}%
• Conservador: VPL R$ {cenarios['conservador']['vpl']:,.2f} | TIR {cenarios['conservador']['tir']:.1f}%
• Pessimista: VPL R$ {cenarios['pessimista']['vpl']:,.2f} | TIR {cenarios['pessimista']['tir']:.1f}%

CONCLUSÃO:
{'✅ INVESTIMENTO VIÁVEL' if resultado_financeiro['vpl'] > 0 else '❌ INVESTIMENTO NÃO VIÁVEL'}

Relatório gerado em: {pd.Timestamp.now().strftime('%d/%m/%Y %H:%M:%S')}
"""

with open('../data/relatorio_financeiro.txt', 'w', encoding='utf-8') as f:
    f.write(relatorio_resumido)

print("💾 Resultados financeiros salvos em:")
print("- ../data/financial_analysis_results.json")
print("- ../data/cash_flow_analysis.csv")
print("- ../data/relatorio_financeiro.txt")
print("\n✅ Análise financeira completa!")
print("\n📋 PRÓXIMOS PASSOS:")
print("1. Revisar os resultados e validar premissas")
print("2. Ajustar parâmetros se necessário")
print("3. Executar análise de Monte Carlo para maior robustez")
print("4. Comparar com outros cenários de investimento")